# Ref-B
* GCR = 0.4
* Albedo = 0.2
* Hub Height = 3.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* For a 2-Up portrait system with a GCR of 0.4 the pitch is 2*5.96m = 11.92m
* 2-up Portrait: 5 rows, each with 25 x 2 = 50 modules (Figure 4)
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/Ref-B-2'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/Ref-B-2


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv
from pathlib import Path

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+255.g34c0e0c.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'Ref-B-2'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1.303 # m
module_y = 2.384 # m. slope we will measure
sensorsy=2
torquetube_diam = 0.15

zgap = 0.02 # m
xgap = 0.002 # m

# SceneDict Parameters
pitch = 11.92 # m
albedo = 0.2
hub_height = 3.5 # m  
nMods = 50 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/Ref-B-2
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [6]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [7]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y, xgap=xgap, zgap=zgap)


Module Name: PVmod
Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [8]:
module.addTorquetube(diameter=torquetube_diam, tubetype='Round', material='Metal_Grey', 
                     axisofrotation=True, visible=True, recompile=True)

Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [9]:
weatherfile = str(Path().resolve().parent.parent / 'Instructions' / 'Phase2_meteo_hourly_psm3format.csv')
weatherfile

'/Users/storopov/Documents/GitHub/bifacial_radiance/docs/development/Instructions/Phase2_meteo_hourly_psm3format.csv'

In [10]:
metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-12-01_0000', endtime='2022-12-31_2359')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [11]:
trackerdict = demo.set1axis(**trackerParams)

In [12]:
demo.gendaylit1axis()

Creating ~281 skyfiles. 
Created 263 skyfiles in /skies/


{'2022-11-01_0630': {'surf_azm': 90.0,
  'surf_tilt': 13.77,
  'theta': -13.77,
  'dni': 497,
  'ghi': 64,
  'dhi': 17,
  'temp_air': 6.0,
  'wind_speed': 1.1,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-11-01_0630.rad'},
 '2022-11-01_0730': {'surf_azm': 90.0,
  'surf_tilt': 35.73,
  'theta': -35.73,
  'dni': 721,
  'ghi': 174,
  'dhi': 27,
  'temp_air': 7.4,
  'wind_speed': 0.3,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-11-01_0730.rad'},
 '2022-11-01_0830': {'surf_azm': 90.0,
  'surf_tilt': 55.0,
  'theta': -55.0,
  'dni': 898,
  'ghi': 372,
  'dhi': 37,
  'temp_air': 9.9,
  'wind_speed': 0.8,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-11-01_0830.rad'},
 '2022-11-01_0930': {'surf_azm': 90.0,
  'surf_tilt': 38.6,
  'theta': -38.6,
  'dni': 967,
  'ghi': 541,
  'dhi': 47,
  'temp_air': 12.6,
  'wind_speed': 2.0,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-11-01_0930.rad'},
 '2022-11-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 18.33,
  'theta': -18.33,
  'dni': 870,
  'ghi': 636,
  'd

In [13]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~263 .rad files for gendaylit 1-axis workflow (this takes a minute..)
263 Radfiles created in /objects/

Making 263 octfiles in root directory.
Created 1axis_2022-11-01_0630.oct
Created 1axis_2022-11-01_0730.oct
Created 1axis_2022-11-01_0830.oct
Created 1axis_2022-11-01_0930.oct
Created 1axis_2022-11-01_1030.oct
Created 1axis_2022-11-01_1130.oct
Created 1axis_2022-11-01_1230.oct
Created 1axis_2022-11-01_1330.oct
Created 1axis_2022-11-01_1430.oct
Created 1axis_2022-11-01_1530.oct
Created 1axis_2022-11-02_0730.oct
Created 1axis_2022-11-02_0830.oct
Created 1axis_2022-11-02_0930.oct
Created 1axis_2022-11-02_1030.oct
Created 1axis_2022-11-02_1130.oct
Created 1axis_2022-11-02_1230.oct
Created 1axis_2022-11-02_1330.oct
Created 1axis_2022-11-02_1430.oct
Created 1axis_2022-11-02_1530.oct
Created 1axis_2022-11-02_1630.oct
Created 1axis_2022-11-03_0630.oct
Created 1axis_2022-11-03_0730.oct
Created 1axis_2022-11-03_0830.oct
Created 1axis_2022-11-03_0930.oct
Created 1axis_2022-11-03_1030.oc

/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2174: Warning: Warning: positive Perez parameter B (= 0.172789), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-11-04_1630.oct
Created 1axis_2022-11-05_0730.oct
Created 1axis_2022-11-05_0830.oct
Created 1axis_2022-11-05_0930.oct
Created 1axis_2022-11-05_1030.oct
Created 1axis_2022-11-05_1130.oct
Created 1axis_2022-11-05_1230.oct
Created 1axis_2022-11-05_1330.oct
Created 1axis_2022-11-05_1430.oct
Created 1axis_2022-11-05_1530.oct
Created 1axis_2022-11-05_1630.oct
Created 1axis_2022-11-06_0630.oct
Created 1axis_2022-11-06_0730.oct
Created 1axis_2022-11-06_0830.oct
Created 1axis_2022-11-06_0930.oct
Created 1axis_2022-11-06_1030.oct
Created 1axis_2022-11-06_1130.oct
Created 1axis_2022-11-06_1230.oct
Created 1axis_2022-11-06_1330.oct
Created 1axis_2022-11-06_1430.oct
Created 1axis_2022-11-06_1530.oct


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2174: Warning: Warning: positive Perez parameter B (= 0.246035), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-11-06_1630.oct
Created 1axis_2022-11-07_0730.oct
Created 1axis_2022-11-07_0830.oct
Created 1axis_2022-11-07_0930.oct
Created 1axis_2022-11-07_1030.oct
Created 1axis_2022-11-07_1130.oct
Created 1axis_2022-11-07_1230.oct
Created 1axis_2022-11-07_1330.oct
Created 1axis_2022-11-07_1430.oct
Created 1axis_2022-11-07_1530.oct
Created 1axis_2022-11-07_1630.oct
Created 1axis_2022-11-08_0730.oct
Created 1axis_2022-11-08_0830.oct
Created 1axis_2022-11-08_0930.oct
Created 1axis_2022-11-08_1030.oct
Created 1axis_2022-11-08_1130.oct
Created 1axis_2022-11-08_1230.oct
Created 1axis_2022-11-08_1330.oct
Created 1axis_2022-11-08_1430.oct
Created 1axis_2022-11-08_1530.oct


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2174: Warning: Warning: positive Perez parameter B (= 0.139933), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-11-08_1630.oct
Created 1axis_2022-11-09_0730.oct
Created 1axis_2022-11-09_0830.oct
Created 1axis_2022-11-09_0930.oct
Created 1axis_2022-11-09_1030.oct
Created 1axis_2022-11-09_1130.oct
Created 1axis_2022-11-09_1230.oct
Created 1axis_2022-11-09_1330.oct
Created 1axis_2022-11-09_1430.oct
Created 1axis_2022-11-09_1530.oct


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2174: Warning: Warning: positive Perez parameter B (= 0.254273), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-11-09_1630.oct
Created 1axis_2022-11-10_0630.oct
Created 1axis_2022-11-10_0730.oct
Created 1axis_2022-11-10_0830.oct
Created 1axis_2022-11-10_0930.oct
Created 1axis_2022-11-10_1030.oct
Created 1axis_2022-11-10_1130.oct
Created 1axis_2022-11-10_1230.oct
Created 1axis_2022-11-10_1330.oct
Created 1axis_2022-11-10_1430.oct
Created 1axis_2022-11-10_1530.oct
Created 1axis_2022-11-11_0730.oct
Created 1axis_2022-11-11_0830.oct
Created 1axis_2022-11-11_0930.oct
Created 1axis_2022-11-11_1030.oct
Created 1axis_2022-11-11_1130.oct
Created 1axis_2022-11-11_1230.oct
Created 1axis_2022-11-11_1330.oct
Created 1axis_2022-11-11_1430.oct
Created 1axis_2022-11-11_1530.oct
Created 1axis_2022-11-12_0730.oct
Created 1axis_2022-11-12_0830.oct
Created 1axis_2022-11-12_0930.oct
Created 1axis_2022-11-12_1030.oct
Created 1axis_2022-11-12_1130.oct
Created 1axis_2022-11-12_1230.oct
Created 1axis_2022-11-12_1330.oct
Created 1axis_2022-11-12_1430.oct
Created 1axis_2022-11-12_1530.oct
Created 1axis_

In [14]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted=13) # Middle Module

Linescan in process: 1axis_2022-11-01_0630_Row3_Module13_Front
Linescan in process: 1axis_2022-11-01_0630_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-01_0630_Row3_Module13.csv
Index: 2022-11-01_0630. Wm2Front: 193.79775. Wm2Back: 7.579090166666667
Linescan in process: 1axis_2022-11-01_0730_Row3_Module13_Front
Linescan in process: 1axis_2022-11-01_0730_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-01_0730_Row3_Module13.csv
Index: 2022-11-01_0730. Wm2Front: 545.64125. Wm2Back: 24.67404
Linescan in process: 1axis_2022-11-01_0830_Row3_Module13_Front
Linescan in process: 1axis_2022-11-01_0830_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-01_0830_Row3_Module13.csv
Index: 2022-11-01_0830. Wm2Front: 761.6077666666665. Wm2Back: 55.28923833333333
Linescan in process: 1axis_2022-11-01_0930_Row3_Module13_Front
Linescan in process: 1axis_2022-11-01_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-01_0930_Row3_Module13.csv
Index: 2022-11-01_0930. Wm2Front: 752.11616666666

Linescan in process: 1axis_2022-11-04_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-04_0930_Row3_Module13.csv
Index: 2022-11-04_0930. Wm2Front: 363.0863166666667. Wm2Back: 49.43947333333334
Linescan in process: 1axis_2022-11-04_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-11-04_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-04_1030_Row3_Module13.csv
Index: 2022-11-04_1030. Wm2Front: 76.902565. Wm2Back: 12.491611666666667
Linescan in process: 1axis_2022-11-04_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-11-04_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-04_1130_Row3_Module13.csv
Index: 2022-11-04_1130. Wm2Front: 104.307. Wm2Back: 15.664211666666667
Linescan in process: 1axis_2022-11-04_1230_Row3_Module13_Front
Linescan in process: 1axis_2022-11-04_1230_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-04_1230_Row3_Module13.csv
Index: 2022-11-04_1230. Wm2Front: 181.16991666666667. Wm2Back: 30.615915
Linescan in process: 1axis_202

Linescan in process: 1axis_2022-11-07_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-07_0930_Row3_Module13.csv
Index: 2022-11-07_0930. Wm2Front: 729.3282333333333. Wm2Back: 74.03151
Linescan in process: 1axis_2022-11-07_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-11-07_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-07_1030_Row3_Module13.csv
Index: 2022-11-07_1030. Wm2Front: 689.27065. Wm2Back: 87.68489500000001
Linescan in process: 1axis_2022-11-07_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-11-07_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-07_1130_Row3_Module13.csv
Index: 2022-11-07_1130. Wm2Front: 565.7685. Wm2Back: 77.27681833333332
Linescan in process: 1axis_2022-11-07_1230_Row3_Module13_Front
Linescan in process: 1axis_2022-11-07_1230_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-07_1230_Row3_Module13.csv
Index: 2022-11-07_1230. Wm2Front: 692.5049666666666. Wm2Back: 85.75366166666666
Linescan in process: 1axis_2022-1

Linescan in process: 1axis_2022-11-10_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-10_0930_Row3_Module13.csv
Index: 2022-11-10_0930. Wm2Front: 720.8404166666667. Wm2Back: 69.09442666666666
Linescan in process: 1axis_2022-11-10_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-11-10_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-10_1030_Row3_Module13.csv
Index: 2022-11-10_1030. Wm2Front: 671.8065666666668. Wm2Back: 83.83110333333335
Linescan in process: 1axis_2022-11-10_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-11-10_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-10_1130_Row3_Module13.csv
Index: 2022-11-10_1130. Wm2Front: 684.59825. Wm2Back: 90.68236999999999
Linescan in process: 1axis_2022-11-10_1230_Row3_Module13_Front
Linescan in process: 1axis_2022-11-10_1230_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-10_1230_Row3_Module13.csv
Index: 2022-11-10_1230. Wm2Front: 715.9637166666666. Wm2Back: 82.82549500000002
Linescan in pro

Linescan in process: 1axis_2022-11-13_1430_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-13_1430_Row3_Module13.csv
Index: 2022-11-13_1430. Wm2Front: 752.3900833333333. Wm2Back: 50.768636666666666
Linescan in process: 1axis_2022-11-13_1530_Row3_Module13_Front
Linescan in process: 1axis_2022-11-13_1530_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-13_1530_Row3_Module13.csv
Index: 2022-11-13_1530. Wm2Front: 390.85785. Wm2Back: 23.407318333333336
Linescan in process: 1axis_2022-11-14_0730_Row3_Module13_Front
Linescan in process: 1axis_2022-11-14_0730_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-14_0730_Row3_Module13.csv
Index: 2022-11-14_0730. Wm2Front: 71.86153. Wm2Back: 12.015125
Linescan in process: 1axis_2022-11-14_0830_Row3_Module13_Front
Linescan in process: 1axis_2022-11-14_0830_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-14_0830_Row3_Module13.csv
Index: 2022-11-14_0830. Wm2Front: 224.042. Wm2Back: 39.410515000000004
Linescan in process: 1axis_2022-11-14_0

Linescan in process: 1axis_2022-11-17_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-17_1030_Row3_Module13.csv
Index: 2022-11-17_1030. Wm2Front: 668.7162833333333. Wm2Back: 82.76491
Linescan in process: 1axis_2022-11-17_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-11-17_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-17_1130_Row3_Module13.csv
Index: 2022-11-17_1130. Wm2Front: 654.48605. Wm2Back: 86.85843333333334
Linescan in process: 1axis_2022-11-17_1230_Row3_Module13_Front
Linescan in process: 1axis_2022-11-17_1230_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-17_1230_Row3_Module13.csv
Index: 2022-11-17_1230. Wm2Front: 689.1698833333332. Wm2Back: 80.28788499999999
Linescan in process: 1axis_2022-11-17_1330_Row3_Module13_Front
Linescan in process: 1axis_2022-11-17_1330_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-17_1330_Row3_Module13.csv
Index: 2022-11-17_1330. Wm2Front: 746.3948666666668. Wm2Back: 64.630045
Linescan in process: 1axis_2022-

Linescan in process: 1axis_2022-11-20_1530_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-20_1530_Row3_Module13.csv
Index: 2022-11-20_1530. Wm2Front: 376.35091666666665. Wm2Back: 24.256905000000003
Linescan in process: 1axis_2022-11-21_0730_Row3_Module13_Front
Linescan in process: 1axis_2022-11-21_0730_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-21_0730_Row3_Module13.csv
Index: 2022-11-21_0730. Wm2Front: 385.74465. Wm2Back: 13.973838333333333
Linescan in process: 1axis_2022-11-21_0830_Row3_Module13_Front
Linescan in process: 1axis_2022-11-21_0830_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-21_0830_Row3_Module13.csv
Index: 2022-11-21_0830. Wm2Front: 697.2814666666667. Wm2Back: 48.476456666666664
Linescan in process: 1axis_2022-11-21_0930_Row3_Module13_Front
Linescan in process: 1axis_2022-11-21_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-21_0930_Row3_Module13.csv
Index: 2022-11-21_0930. Wm2Front: 692.1599333333334. Wm2Back: 65.373265
Linescan in process

Linescan in process: 1axis_2022-11-24_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-24_1130_Row3_Module13.csv
Index: 2022-11-24_1130. Wm2Front: 380.85659999999996. Wm2Back: 56.18638333333334
Linescan in process: 1axis_2022-11-24_1230_Row3_Module13_Front
Linescan in process: 1axis_2022-11-24_1230_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-24_1230_Row3_Module13.csv
Index: 2022-11-24_1230. Wm2Front: 495.62321666666674. Wm2Back: 65.62738333333333
Linescan in process: 1axis_2022-11-24_1330_Row3_Module13_Front
Linescan in process: 1axis_2022-11-24_1330_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-24_1330_Row3_Module13.csv
Index: 2022-11-24_1330. Wm2Front: 340.65338333333335. Wm2Back: 57.70834000000001
Linescan in process: 1axis_2022-11-24_1430_Row3_Module13_Front
Linescan in process: 1axis_2022-11-24_1430_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-24_1430_Row3_Module13.csv
Index: 2022-11-24_1430. Wm2Front: 149.61038333333335. Wm2Back: 37.31628666666667
Lin

Linescan in process: 1axis_2022-11-28_0730_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-28_0730_Row3_Module13.csv
Index: 2022-11-28_0730. Wm2Front: 325.7984166666667. Wm2Back: 19.161163333333334
Linescan in process: 1axis_2022-11-28_0830_Row3_Module13_Front
Linescan in process: 1axis_2022-11-28_0830_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-28_0830_Row3_Module13.csv
Index: 2022-11-28_0830. Wm2Front: 284.12519999999995. Wm2Back: 43.110960000000006
Linescan in process: 1axis_2022-11-28_0930_Row3_Module13_Front
Linescan in process: 1axis_2022-11-28_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-28_0930_Row3_Module13.csv
Index: 2022-11-28_0930. Wm2Front: 295.0250666666667. Wm2Back: 48.929595
Linescan in process: 1axis_2022-11-28_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-11-28_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-11-28_1030_Row3_Module13.csv
Index: 2022-11-28_1030. Wm2Front: 237.44391666666667. Wm2Back: 38.44484833333334
Linescan in

In [15]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 1) # South Module

Linescan in process: 1axis_2022-11-01_0630_Row3_Module1_Front
Linescan in process: 1axis_2022-11-01_0630_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-01_0630_Row3_Module1.csv
Index: 2022-11-01_0630. Wm2Front: 194.35176666666666. Wm2Back: 16.657356666666665
Linescan in process: 1axis_2022-11-01_0730_Row3_Module1_Front
Linescan in process: 1axis_2022-11-01_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-01_0730_Row3_Module1.csv
Index: 2022-11-01_0730. Wm2Front: 548.7873166666666. Wm2Back: 43.02113333333334
Linescan in process: 1axis_2022-11-01_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-11-01_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-01_0830_Row3_Module1.csv
Index: 2022-11-01_0830. Wm2Front: 769.6711333333334. Wm2Back: 72.39239666666666
Linescan in process: 1axis_2022-11-01_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-11-01_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-01_0930_Row3_Module1.csv
Index: 2022-11-01_0930. Wm2Front: 

Linescan in process: 1axis_2022-11-04_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-04_0930_Row3_Module1.csv
Index: 2022-11-04_0930. Wm2Front: 367.4369833333334. Wm2Back: 61.81856500000001
Linescan in process: 1axis_2022-11-04_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-11-04_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-04_1030_Row3_Module1.csv
Index: 2022-11-04_1030. Wm2Front: 77.06099333333334. Wm2Back: 14.032824999999999
Linescan in process: 1axis_2022-11-04_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-11-04_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-04_1130_Row3_Module1.csv
Index: 2022-11-04_1130. Wm2Front: 104.2173. Wm2Back: 17.756463333333336
Linescan in process: 1axis_2022-11-04_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-11-04_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-04_1230_Row3_Module1.csv
Index: 2022-11-04_1230. Wm2Front: 182.1325333333333. Wm2Back: 35.03689333333334
Linescan in process: 1axi

Linescan in process: 1axis_2022-11-07_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-07_0930_Row3_Module1.csv
Index: 2022-11-07_0930. Wm2Front: 733.44595. Wm2Back: 101.78431666666668
Linescan in process: 1axis_2022-11-07_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-11-07_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-07_1030_Row3_Module1.csv
Index: 2022-11-07_1030. Wm2Front: 690.1801833333333. Wm2Back: 122.3698
Linescan in process: 1axis_2022-11-07_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-11-07_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-07_1130_Row3_Module1.csv
Index: 2022-11-07_1130. Wm2Front: 565.8917833333333. Wm2Back: 105.87668333333332
Linescan in process: 1axis_2022-11-07_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-11-07_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-07_1230_Row3_Module1.csv
Index: 2022-11-07_1230. Wm2Front: 695.3139666666667. Wm2Back: 116.54419999999999
Linescan in process: 1axis_2022-

Linescan in process: 1axis_2022-11-10_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-10_0930_Row3_Module1.csv
Index: 2022-11-10_0930. Wm2Front: 724.2219666666667. Wm2Back: 96.49775666666667
Linescan in process: 1axis_2022-11-10_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-11-10_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-10_1030_Row3_Module1.csv
Index: 2022-11-10_1030. Wm2Front: 671.8804666666666. Wm2Back: 118.86126666666667
Linescan in process: 1axis_2022-11-10_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-11-10_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-10_1130_Row3_Module1.csv
Index: 2022-11-10_1130. Wm2Front: 684.5362833333334. Wm2Back: 129.23886666666667
Linescan in process: 1axis_2022-11-10_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-11-10_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-10_1230_Row3_Module1.csv
Index: 2022-11-10_1230. Wm2Front: 717.0739333333333. Wm2Back: 117.09658333333334
Linescan in pro

Linescan in process: 1axis_2022-11-13_1430_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-13_1430_Row3_Module1.csv
Index: 2022-11-13_1430. Wm2Front: 762.1531166666666. Wm2Back: 62.883008333333336
Linescan in process: 1axis_2022-11-13_1530_Row3_Module1_Front
Linescan in process: 1axis_2022-11-13_1530_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-13_1530_Row3_Module1.csv
Index: 2022-11-13_1530. Wm2Front: 392.23215000000005. Wm2Back: 33.02551666666666
Linescan in process: 1axis_2022-11-14_0730_Row3_Module1_Front
Linescan in process: 1axis_2022-11-14_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-14_0730_Row3_Module1.csv
Index: 2022-11-14_0730. Wm2Front: 72.37186166666666. Wm2Back: 13.741541666666667
Linescan in process: 1axis_2022-11-14_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-11-14_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-14_0830_Row3_Module1.csv
Index: 2022-11-14_0830. Wm2Front: 230.87240000000003. Wm2Back: 45.708685
Linescan in process: 1

Linescan in process: 1axis_2022-11-17_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-17_1030_Row3_Module1.csv
Index: 2022-11-17_1030. Wm2Front: 669.0947666666666. Wm2Back: 117.32588333333334
Linescan in process: 1axis_2022-11-17_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-11-17_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-17_1130_Row3_Module1.csv
Index: 2022-11-17_1130. Wm2Front: 654.3550166666666. Wm2Back: 124.17421666666667
Linescan in process: 1axis_2022-11-17_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-11-17_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-17_1230_Row3_Module1.csv
Index: 2022-11-17_1230. Wm2Front: 690.1942166666665. Wm2Back: 113.41489999999999
Linescan in process: 1axis_2022-11-17_1330_Row3_Module1_Front
Linescan in process: 1axis_2022-11-17_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-17_1330_Row3_Module1.csv
Index: 2022-11-17_1330. Wm2Front: 752.2226833333333. Wm2Back: 88.12851999999998
Linescan in pro

Linescan in process: 1axis_2022-11-20_1530_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-20_1530_Row3_Module1.csv
Index: 2022-11-20_1530. Wm2Front: 377.41315000000003. Wm2Back: 31.574135
Linescan in process: 1axis_2022-11-21_0730_Row3_Module1_Front
Linescan in process: 1axis_2022-11-21_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-21_0730_Row3_Module1.csv
Index: 2022-11-21_0730. Wm2Front: 387.29156666666665. Wm2Back: 31.850736666666663
Linescan in process: 1axis_2022-11-21_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-11-21_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-21_0830_Row3_Module1.csv
Index: 2022-11-21_0830. Wm2Front: 705.4316. Wm2Back: 61.44809166666667
Linescan in process: 1axis_2022-11-21_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-11-21_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-21_0930_Row3_Module1.csv
Index: 2022-11-21_0930. Wm2Front: 695.7934333333334. Wm2Back: 90.17570166666667
Linescan in process: 1axis_2022-

Linescan in process: 1axis_2022-11-24_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-24_1130_Row3_Module1.csv
Index: 2022-11-24_1130. Wm2Front: 380.9049. Wm2Back: 68.21539999999999
Linescan in process: 1axis_2022-11-24_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-11-24_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-24_1230_Row3_Module1.csv
Index: 2022-11-24_1230. Wm2Front: 497.34826666666663. Wm2Back: 85.68381
Linescan in process: 1axis_2022-11-24_1330_Row3_Module1_Front
Linescan in process: 1axis_2022-11-24_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-24_1330_Row3_Module1.csv
Index: 2022-11-24_1330. Wm2Front: 346.9159. Wm2Back: 68.47057
Linescan in process: 1axis_2022-11-24_1430_Row3_Module1_Front
Linescan in process: 1axis_2022-11-24_1430_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-24_1430_Row3_Module1.csv
Index: 2022-11-24_1430. Wm2Front: 153.76668333333333. Wm2Back: 41.67337166666666
Linescan in process: 1axis_2022-11-24_1530_Row3_Modu

Saved: results/irr_1axis_2022-11-28_0730_Row3_Module1.csv
Index: 2022-11-28_0730. Wm2Front: 328.06038333333333. Wm2Back: 30.022058333333334
Linescan in process: 1axis_2022-11-28_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-11-28_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-28_0830_Row3_Module1.csv
Index: 2022-11-28_0830. Wm2Front: 291.653. Wm2Back: 50.05474
Linescan in process: 1axis_2022-11-28_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-11-28_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-28_0930_Row3_Module1.csv
Index: 2022-11-28_0930. Wm2Front: 301.0383. Wm2Back: 58.26215166666667
Linescan in process: 1axis_2022-11-28_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-11-28_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-11-28_1030_Row3_Module1.csv
Index: 2022-11-28_1030. Wm2Front: 238.5313. Wm2Back: 44.276390000000006
Linescan in process: 1axis_2022-11-28_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-11-28_1130_Row3_Mod

In [16]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 25) # North Module

Linescan in process: 1axis_2022-11-01_0630_Row3_Module25_Front
Linescan in process: 1axis_2022-11-01_0630_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-01_0630_Row3_Module25.csv
Index: 2022-11-01_0630. Wm2Front: 194.01295000000002. Wm2Back: 7.090596833333334
Linescan in process: 1axis_2022-11-01_0730_Row3_Module25_Front
Linescan in process: 1axis_2022-11-01_0730_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-01_0730_Row3_Module25.csv
Index: 2022-11-01_0730. Wm2Front: 545.7725833333334. Wm2Back: 23.55829333333333
Linescan in process: 1axis_2022-11-01_0830_Row3_Module25_Front
Linescan in process: 1axis_2022-11-01_0830_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-01_0830_Row3_Module25.csv
Index: 2022-11-01_0830. Wm2Front: 760.7556833333333. Wm2Back: 54.131936666666675
Linescan in process: 1axis_2022-11-01_0930_Row3_Module25_Front
Linescan in process: 1axis_2022-11-01_0930_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-01_0930_Row3_Module25.csv
Index: 2022-11-01_0930

Saved: results/irr_1axis_2022-11-04_0830_Row3_Module25.csv
Index: 2022-11-04_0830. Wm2Front: 79.65811166666667. Wm2Back: 24.732615
Linescan in process: 1axis_2022-11-04_0930_Row3_Module25_Front
Linescan in process: 1axis_2022-11-04_0930_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-04_0930_Row3_Module25.csv
Index: 2022-11-04_0930. Wm2Front: 362.71576666666664. Wm2Back: 48.70333333333333
Linescan in process: 1axis_2022-11-04_1030_Row3_Module25_Front
Linescan in process: 1axis_2022-11-04_1030_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-04_1030_Row3_Module25.csv
Index: 2022-11-04_1030. Wm2Front: 76.90580833333334. Wm2Back: 12.409953333333334
Linescan in process: 1axis_2022-11-04_1130_Row3_Module25_Front
Linescan in process: 1axis_2022-11-04_1130_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-04_1130_Row3_Module25.csv
Index: 2022-11-04_1130. Wm2Front: 104.26415. Wm2Back: 15.606386666666666
Linescan in process: 1axis_2022-11-04_1230_Row3_Module25_Front
Linescan in process

Linescan in process: 1axis_2022-11-07_0830_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-07_0830_Row3_Module25.csv
Index: 2022-11-07_0830. Wm2Front: 615.3807666666667. Wm2Back: 55.129265
Linescan in process: 1axis_2022-11-07_0930_Row3_Module25_Front
Linescan in process: 1axis_2022-11-07_0930_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-07_0930_Row3_Module25.csv
Index: 2022-11-07_0930. Wm2Front: 729.1171666666667. Wm2Back: 73.81489666666667
Linescan in process: 1axis_2022-11-07_1030_Row3_Module25_Front
Linescan in process: 1axis_2022-11-07_1030_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-07_1030_Row3_Module25.csv
Index: 2022-11-07_1030. Wm2Front: 688.6489333333333. Wm2Back: 87.14442666666667
Linescan in process: 1axis_2022-11-07_1130_Row3_Module25_Front
Linescan in process: 1axis_2022-11-07_1130_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-07_1130_Row3_Module25.csv
Index: 2022-11-07_1130. Wm2Front: 566.24935. Wm2Back: 77.21986166666666
Linescan in process: 1a

Linescan in process: 1axis_2022-11-10_0830_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-10_0830_Row3_Module25.csv
Index: 2022-11-10_0830. Wm2Front: 726.4419333333333. Wm2Back: 50.049009999999996
Linescan in process: 1axis_2022-11-10_0930_Row3_Module25_Front
Linescan in process: 1axis_2022-11-10_0930_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-10_0930_Row3_Module25.csv
Index: 2022-11-10_0930. Wm2Front: 720.3611166666667. Wm2Back: 68.39013166666666
Linescan in process: 1axis_2022-11-10_1030_Row3_Module25_Front
Linescan in process: 1axis_2022-11-10_1030_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-10_1030_Row3_Module25.csv
Index: 2022-11-10_1030. Wm2Front: 671.2243166666667. Wm2Back: 83.20440500000001
Linescan in process: 1axis_2022-11-10_1130_Row3_Module25_Front
Linescan in process: 1axis_2022-11-10_1130_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-10_1130_Row3_Module25.csv
Index: 2022-11-10_1130. Wm2Front: 684.7218499999999. Wm2Back: 90.313075
Linescan in pr

Linescan in process: 1axis_2022-11-13_1330_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-13_1330_Row3_Module25.csv
Index: 2022-11-13_1330. Wm2Front: 733.4405666666667. Wm2Back: 63.94884333333333
Linescan in process: 1axis_2022-11-13_1430_Row3_Module25_Front
Linescan in process: 1axis_2022-11-13_1430_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-13_1430_Row3_Module25.csv
Index: 2022-11-13_1430. Wm2Front: 751.90375. Wm2Back: 49.58415
Linescan in process: 1axis_2022-11-13_1530_Row3_Module25_Front
Linescan in process: 1axis_2022-11-13_1530_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-13_1530_Row3_Module25.csv
Index: 2022-11-13_1530. Wm2Front: 390.33621666666664. Wm2Back: 22.39492666666667
Linescan in process: 1axis_2022-11-14_0730_Row3_Module25_Front
Linescan in process: 1axis_2022-11-14_0730_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-14_0730_Row3_Module25.csv
Index: 2022-11-14_0730. Wm2Front: 71.75100833333333. Wm2Back: 11.930061666666667
Linescan in process: 1

Linescan in process: 1axis_2022-11-17_0930_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-17_0930_Row3_Module25.csv
Index: 2022-11-17_0930. Wm2Front: 708.7950500000001. Wm2Back: 66.30394333333334
Linescan in process: 1axis_2022-11-17_1030_Row3_Module25_Front
Linescan in process: 1axis_2022-11-17_1030_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-17_1030_Row3_Module25.csv
Index: 2022-11-17_1030. Wm2Front: 668.3304. Wm2Back: 81.69488833333332
Linescan in process: 1axis_2022-11-17_1130_Row3_Module25_Front
Linescan in process: 1axis_2022-11-17_1130_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-17_1130_Row3_Module25.csv
Index: 2022-11-17_1130. Wm2Front: 654.6667666666667. Wm2Back: 86.42129166666668
Linescan in process: 1axis_2022-11-17_1230_Row3_Module25_Front
Linescan in process: 1axis_2022-11-17_1230_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-17_1230_Row3_Module25.csv
Index: 2022-11-17_1230. Wm2Front: 688.527. Wm2Back: 79.60646
Linescan in process: 1axis_2022-11-

Linescan in process: 1axis_2022-11-20_1430_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-20_1430_Row3_Module25.csv
Index: 2022-11-20_1430. Wm2Front: 771.6617666666667. Wm2Back: 48.006535
Linescan in process: 1axis_2022-11-20_1530_Row3_Module25_Front
Linescan in process: 1axis_2022-11-20_1530_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-20_1530_Row3_Module25.csv
Index: 2022-11-20_1530. Wm2Front: 376.36716666666666. Wm2Back: 23.258135000000003
Linescan in process: 1axis_2022-11-21_0730_Row3_Module25_Front
Linescan in process: 1axis_2022-11-21_0730_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-21_0730_Row3_Module25.csv
Index: 2022-11-21_0730. Wm2Front: 385.98951666666665. Wm2Back: 13.02292
Linescan in process: 1axis_2022-11-21_0830_Row3_Module25_Front
Linescan in process: 1axis_2022-11-21_0830_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-21_0830_Row3_Module25.csv
Index: 2022-11-21_0830. Wm2Front: 696.2768833333333. Wm2Back: 47.48530000000001
Linescan in process: 

Linescan in process: 1axis_2022-11-24_1030_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-24_1030_Row3_Module25.csv
Index: 2022-11-24_1030. Wm2Front: 383.7654333333333. Wm2Back: 59.12352333333334
Linescan in process: 1axis_2022-11-24_1130_Row3_Module25_Front
Linescan in process: 1axis_2022-11-24_1130_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-24_1130_Row3_Module25.csv
Index: 2022-11-24_1130. Wm2Front: 380.945. Wm2Back: 55.86761666666667
Linescan in process: 1axis_2022-11-24_1230_Row3_Module25_Front
Linescan in process: 1axis_2022-11-24_1230_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-24_1230_Row3_Module25.csv
Index: 2022-11-24_1230. Wm2Front: 495.30878333333334. Wm2Back: 64.78879
Linescan in process: 1axis_2022-11-24_1330_Row3_Module25_Front
Linescan in process: 1axis_2022-11-24_1330_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-24_1330_Row3_Module25.csv
Index: 2022-11-24_1330. Wm2Front: 339.4160166666667. Wm2Back: 56.95112833333334
Linescan in process: 1axi

Linescan in process: 1axis_2022-11-27_1530_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-27_1530_Row3_Module25.csv
Index: 2022-11-27_1530. Wm2Front: 344.89865. Wm2Back: 20.606221666666666
Linescan in process: 1axis_2022-11-28_0730_Row3_Module25_Front
Linescan in process: 1axis_2022-11-28_0730_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-28_0730_Row3_Module25.csv
Index: 2022-11-28_0730. Wm2Front: 325.73081666666667. Wm2Back: 18.19251333333333
Linescan in process: 1axis_2022-11-28_0830_Row3_Module25_Front
Linescan in process: 1axis_2022-11-28_0830_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-28_0830_Row3_Module25.csv
Index: 2022-11-28_0830. Wm2Front: 282.9014166666667. Wm2Back: 42.71159833333333
Linescan in process: 1axis_2022-11-28_0930_Row3_Module25_Front
Linescan in process: 1axis_2022-11-28_0930_Row3_Module25_Back
Saved: results/irr_1axis_2022-11-28_0930_Row3_Module25.csv
Index: 2022-11-28_0930. Wm2Front: 294.57245. Wm2Back: 48.09513333333334
Linescan in process: 

In [17]:
demo.calculateResults()

Bifaciality factor of module stored is  1
No CECModule data passed; using default for Prism Solar BHC72-400


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

{'2022-11-01_0630': {'surf_azm': 90.0,
  'surf_tilt': 13.77,
  'theta': -13.77,
  'dni': 497,
  'ghi': 64,
  'dhi': 17,
  'temp_air': 6.0,
  'wind_speed': 1.1,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-11-01_0630.rad',
  'radfile': 'objects/1axis2022-11-01_0630__C_3.23889_rtr_11.92000_tilt_13.77000_50modsx5rows_origin0,0.rad',
  'scene': {'module': {'x': 1.303, 'y': 2.384, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.305, 'sceney': 2.384, 'scenez': 0.095, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmod 1.303 2.384 0.02 | xform -t -0.6515 -1.192 0.095 -a 1 -t 0 2.384 0\r\n! genrev Metal_Grey tube1 t*1.305 0.075 32 | xform -ry 90 -t -0.6525 0 0', 'modulefile': 'objects/PVmod.rad', 'glass': False, 'offsetfromaxis': 0.095, 'xgap': 0.002, 'ygap': 0.0, 'zgap': 0.02}, 'modulefile': 'objects/PVmod.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.19999999999999998, 'text': '!xform -rx 13.77 -t 0 0 3.5 -a 50 -t 1.305 0 0 -a 5 -t 0 11.92 0 -i 1 -t -31.32 -23.84 0 -rz 90.0 -t 0 0 0 o

In [18]:
demo.CompiledResults.set_index('timestamp', inplace=True)
demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

,module,row,POA_eff,Grear_mean,Gfront_mean,Module_temp,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI
timestamp,,,,,,,,,,,,,,,,
2022-11-01_0630,13,3,201.376840,7.579090,193.797750,11.877545,83.410302,80.251127,3.910825,3.936612,0.000150,83.410177,1.1,497,17,64
2022-11-01_0630,1,3,211.009123,16.657357,194.351767,12.158680,87.420483,80.482165,8.570726,8.620938,0.000039,87.420449,1.1,497,17,64
2022-11-01_0630,25,3,201.103547,7.090597,194.012950,11.869568,83.296443,80.340873,3.654703,3.678788,0.000140,83.296326,1.1,497,17,64
2022-11-01_0730,13,3,570.315290,24.674040,545.641250,24.969188,229.393277,220.035429,4.522026,4.252882,0.000226,229.392759,0.3,721,27,174
2022-11-01_0730,1,3,591.808450,43.021133,548.787317,25.631308,237.488386,221.232426,7.839309,7.347910,0.000013,237.488354,0.3,721,27,174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-28_1430,1,3,319.316215,53.783515,265.532700,18.257653,130.643216,108.775561,20.254950,20.103464,0.000449,130.642630,2.2,53,198,221
2022-11-28_1430,25,3,302.146922,45.873222,256.273700,17.792140,123.686577,104.989432,17.900090,17.808598,0.000574,123.685867,2.2,53,198,221
2022-11-28_1530,13,3,370.679418,28.677568,342.001850,21.717195,150.158388,138.728367,8.385209,8.239137,0.000109,150.158224,2.4,602,97,241


In [19]:
demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'Ref-B-Nov.csv')

In [20]:
demo.CompiledResults.to_csv(Path().resolve().parent.parent / 'Results' / 'Ref-B-all-Nov.csv')